# Práctica Funciones Splines

**Nombre:** Jesús Muñoz Velasco

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

El objetivo final de esta práctica es construir el gráfico que ilustra el fenómeno de Runge, pero utilizando ahora un spline cúbico natural.

Se desea construir un gráfico en el que se incluyan la función interpolada $f(x) = \dfrac{1}{1+x^2}$
 y las gráficas de dos splines cúbicos naturales que interpolan a f
 respectivamente 11 y 21 en puntos equiespaciados del intervalo $[-5,5]$.

**Ejercicio 1:** Construya un array que contenga un conjunto de n  puntos equiespaciados del intervalo $[-5,5]$.

In [ ]:
n=11
lista_x = np.linspace(-5,5,n)

**Ejercicio 2:** Construya un array que contenga las imágenes mediante f de los puntos anteriores.

In [ ]:
def f(x):
    return (1/(1+x**2))

lista_f=f(lista_x)

**Ejercicio 3:** Construya una función en Python que a partir de los arrays anteriores  debe construir una función spline que obtenga la lista de polinomios que constituyen el **spline** usando el sistema tridiagonal de ecuaciones. 

In [ ]:
def sust_regresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución regresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular superior.
    """
    # A=A.astype(float)     # Con esto se cambian los elementos a tipo flotante pero no es necesario
    # b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema
                       
    # ahora se resuelve el sistema por sustitución regresiva
    x[n-1]=b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):     
        x[i]=(b[i]-np.dot(A[i][i+1:],x[i+1:]))/A[i,i]
                
    return x

def sust_progresiva(A,b):   #función del ejercicio 1
    """
    Esta función resuelve el sistema Ax=b mediante sustitución progresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular inferior.
    """
    # A=A.astype(float)     # Con esto se cambian los elementos a tipo flotante pero no es necesario
    # b=b.astype(float) 
    x=np.zeros(np.size(b))         # almacenaremos aquí la solución del sistema
                       
    # ahora se resuelve el sistema por sustitución progresiva
    x[0]=b[0]/A[0,0]
    for i in range(np.size(b)):     
        x[i]=(b[i]-np.dot(A[i][:i],x[:i]))/A[i,i]
                
    return x

def Cholesky (A):
    """
    Esta función realiza la factorización de Cholesky que consiste en 
    A = L1 * L2
    donde A es simétrica, L1 es triangular inferior y L2 es L1 traspuesta
    """
    n=np.shape(A)[0] #toma el valor del número de filas de A
    L=np.zeros([n,n])
    
    for k in range (n):
        
        sumatory1=0
        
        for s in range (k):
            sumatory1+=((L[k][s])**2)
        
        L[k][k]=((A[k][k]-sumatory1)**0.5)
        
        for j in range (k+1,n):
            sumatory2=0
            
            for s in range (k):
                sumatory2+=(L[j][s]*L[k][s])
            
            L[j][k]=(A[j][k]-sumatory2)/L[k][k]
            
    Lt=np.transpose(L)
            
    return L,Lt


def spline(lista_x, lista_f):
    
    n=len(lista_x)
    h=np.zeros([n])
    delta=np.zeros([n])
    
    for i in range (n-1):
        h[i]=(lista_x[i+1] - lista_x[i])
        delta[i]=((lista_f[i+1]-lista_f[i])/h[i])
    
    matriz=np.zeros([n,n])
    ter_ind=np.zeros([n])
    
    matriz[0][0]=(2/h[0])
    matriz[0][1]=(1/h[0])
    
    matriz[n-1][n-2]=(1/h[n-2])
    matriz[n-1][n-1]=(2/h[n-2])
    
    ter_ind[0]=3*(delta[0]/h[0])
    ter_ind[n-1]=3*(delta[n-2]/h[n-2])
    
    
    for i in range (1,n-1):
        matriz[i][i-1]=(1/h[i-1])
        matriz[i][i+1]=(1/h[i])
        matriz[i][i]= 2*(matriz[i][i-1]+matriz[i][i+1])
        
        ter_ind[i]=3*((delta[i-1]/h[i-1])+(delta[i]/h[i]))
        
    L,Lt=Cholesky(matriz)
    
    y=sust_progresiva(L,ter_ind) # Resuelvo el sistema triangular inferior
    x=sust_regresiva(Lt,y)  # Resuelvo el sistema triangular superior
    
    
    return x



**Ejercicio 4:** Para dibujar la lista de polinomios que constituyen el spline construya su propia función **evalspline** que dados los puntos de interpolación, sus imágenes,la lista de derivadas devuelta por la función **spline**, y un punto $x$ arbitrario del intervalo $[-5,5]$ proporcione el valor del spline en el punto $x$. Para ello, primero deberá localizar el intervalo en que se encuentra $x$ y después deberá evaluar en $x$ el polinomio de grado 3 que define el spline en dicho intervalo.

In [ ]:
def intervalo(lista_x , punto):
    """
    Dado un punto que supongo que está entre dos puntos de una lista ordenada me indica el punto inmediatamente anterior
    """
    i=0
    while (lista_x[i]<punto):
        i=i+1
    
    return (i-1)

def Hermite(x,y,lista_d,lista_x):
    """
    Función que calcula los coeficientes para la función de Hermite
    """
    n=2
    i=intervalo(lista_x,((x[0]+x[1])/2))
        
    D=np.zeros([2*n]) #creo un array vacío
    
    D[0]=y[0]
    D[1]=lista_d[i]

    delta=((y[1]-y[0])/(x[1]-x[0]))
    
    D[2]=(delta-D[1])/(x[1]-x[0])
    
    aux=((lista_d[i+1]-delta)/(x[1]-x[0]))
    
    D[3]=((aux-D[2])/(x[1]-x[0]))
    
    return (D)


def evaluar(x, hermite, t):
    """
    Función que evalúa el punto t que se encuentra en el intervalo que define x
    """
    return(hermite[0]+hermite[1]*(t-x[0])+hermite[2]*(t-x[0])**2+hermite[3]*((t-x[0])**2)*(t-x[1]))

def contenido(lista,punto):
    """
    Función que comprueba si un punto se encuentra o no en una lista 
        - En caso afirmativo devolverá 1 y la posición
        - En caso negativo devolverá 0 y el tamaño de la lista
    """
    found=0
    i=0
    while ((found==0) and (i<len(lista))):
        if (punto==lista[i]):
            found=1
        else:
            i=i+1
        
    return found,i
        

def evalspline(x, y, d, punto):
    """
    Función que dada una lista de puntos y sus imágenes, la lista de derivadas y un punto proporciona el valor del spline
    en el punto.
    """
    cont,posicion=contenido(x,punto)

    if(cont==1): #Si el punto está en la lista
        valor=y[posicion]
    else:
        #Evalúo la función dentro de su intervalo 
        
        intervalo_x=np.array([x[intervalo(x,punto)],x[intervalo(x,punto)+1]])
        intervalo_y=np.array([y[intervalo(x,punto)],y[intervalo(x,punto)+1]])
        
        hermite=Hermite(intervalo_x,intervalo_y,d,x)
        valor=evaluar(intervalo_x,hermite,punto)

    return valor
    

**Ejercicio 5:** Utilice la función anterior para construir la gráfica pedida.

In [ ]:
v2=500 #Número de puntos para la representación
array=np.linspace(-5,5,v2)

n1=11
x1=np.linspace(-5,5,n1) #vector creado para el cálculo del spline de 11 puntos
y1=f(x1)

n2=21
x2=np.linspace(-5,5,n2) #vector creado para el cálculo del spline de 21 puntos
y2=f(x2)

valores1=[evalspline(x1,y1,spline(x1,y1),t) for t in array]
valores2=[evalspline(x2,y2,spline(x2,y2),t) for t in array]

plt.figure(figsize=(9,6))
plt.title("Splines cúbicos")
plt.plot(array,f(array),'r',label="$p(x)=\dfrac{1}{1+x^2}$") #función dada por el enunciado

plt.plot(array,valores1,'blue',label= f"$s_{1}(x)\,\,\,\, ({n1} \,\, puntos)$")
plt.plot(array,valores2,'violet',label= f"$s_{2}(x)\,\,\,\, ({n2} \,\, puntos)$")

plt.legend()
plt.grid()
plt.ylim(-1,2)
plt.xlim(-6,6)
plt.plot([0,0],[-1,2],'k')
plt.plot([-6,6],[0,0],'k')

plt.show()
    